<a href="https://colab.research.google.com/github/desmond-rn/projet-inverse-2d/blob/master/src/notebook/format_binaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture du format binaire SDS

In [ ]:
%reset -f

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import keras
import random
import sklearn
import struct

from ast import literal_eval as l_eval

In [ ]:
from keras import layers
from keras import models
from keras import utils 
from keras import losses
from keras import metrics
from keras import optimizers
from keras import backend as K
from sklearn import metrics

In [ ]:
np.set_printoptions(precision = 3)

## Chargement des données

In [ ]:
""" DEPUIS GOOGLE DRIVE """

# from google.colab import drive
# drive.mount('/gdrive')

' DEPUIS GOOGLE DRIVE '

In [ ]:
%cd ~
%cd /gdrive/My\ Drive
%cd Colab\ Notebooks/Projet\ inverse/data
print()

%ls

/root
/gdrive/My Drive
/gdrive/My Drive/Colab Notebooks/Projet inverse/data

df_rho_part10.pqt  df_rho_part15.pqt  df_rho_part1.pqt   df_rho_part5.pqt
df_rho_part11.pqt  df_rho_part16.pqt  df_rho_part20.pqt  df_rho_part6.pqt
df_rho_part12.pqt  df_rho_part17.pqt  df_rho_part2.pqt   obstacle.png
df_rho_part13.pqt  df_rho_part18.pqt  df_rho_part3.pqt   part_1.sds
df_rho_part14.pqt  df_rho_part19.pqt  df_rho_part4.pqt


Les donnees sont souvent sauvegardées dans un format binaire intitulé **SDS** (source-densité-signal). La version 01 de ce format est structurée dans l'ordre ci-après (l'en-tete du fichier est compris entre les lignes horizontales, et le reste represente la structure d'une des simulations presente dans le fichier):

---
- **`5`** octets pour la constante magique **sds01**
- __`2`__ octets pour le nombre de simulations presentes dans le fichier
- __`2`__ octets pour `N` (nombre de mailles en horizontale)
- __`2`__ octets pour `M` (nombre de mailles en verticale)
- __`2`__ octets pour `step_count` (nombre d'iterations en temps)
- **`1`** octet pour le charactere 0xA (newline)
---
- **`1`** octet pour le bord sur lequel se trouve la source: (0 pour up, 1 pour down, 2 pour left, 3 pour right, et -1 si aucun de ces 4)
- **`1`** octet pour la position de la source sur son bord: (0 pour (0.1, 0.3), 1 pour (0.3, 0.5), 2 pour (0.5, 0.7), 3 pour (0.7, 0.9), et -1 si aucun de ces 4)
- __`4 * 4`__ octets pour les attributs du crenau de densite: 4 octets (float) pour l'abcisse, 4 pour l'ordonee, 4 pour la valeur de la densite en dehors du crenau, et 4 pour la  hauteur du crenau.
- **`step_count * N * 4`** octets pour E_up
- idem pour F_up
- idem pour T_up
- **`step_count * N * 4`** octets pour E_down
- idem pour F_up
- idem pour T_up
- **`step_count * M * 4`** octets pour E_left
- idem pour F_left
- idem pour T_left
- **`step_count * M * 4`** octets pour E_right
- idem pour F_right
- idem pour T_right
- **`1`** octet pour le charactere 0xA (newline)

In [ ]:
def read_sds_version01(file_path):

  file = open(file_path, "rb")

  sds_version = file.read(5)
  assert(sds_version == b'sds01')

  simu_count = int.from_bytes(file.read(2), byteorder='little', signed=False)
  N = int.from_bytes(file.read(2), byteorder='little', signed=False)
  M = int.from_bytes(file.read(2), byteorder='little', signed=False)
  step_count = int.from_bytes(file.read(2), byteorder='little', signed=False)

  assert(file.read(1) == b'\n')

  X = np.zeros(shape=(simu_count // 4, step_count, N, 4, 4, 3), dtype=np.float32)
  y = np.zeros(shape=(simu_count // 4, 4), dtype=np.float32)

  index = 0
  pos_count = 0
  while True:
    byte = file.read(1)
    if byte != True:
      break

    source_edge = int.from_bytes(byte, byteorder='little', signed=True)
    source_pos = int.from_bytes(file.read(1), byteorder='little', signed=True)
    assert (source_edge != -1 and source_pos != -1)
    print("index", index)
    print("source ", source_edge, source_pos)

    rho_attr = file.read(4*4)
    y[index, :] = np.frombuffer(rho_attr, dtype=np.float32)

    for e in range(4):
      for ch in range(3):
        signal = file.read(step_count*N*4)
        X[index, :, :, e, source_pos, ch] = np.frombuffer(signal, dtype=np.float32).reshape(step_count, N)

    assert(file.read(1) == b'\n')
    pos_count += 1
    if pos_count == 4:
      index += 1
      pos_count = 0

  return X, y

In [ ]:
X, y = read_sds_version01("part_1.sds")

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (1, 90, 20, 4, 4, 3)
y shape: (1, 4)


# Calcul du corefficient de determination R^2

Cette fonction est necessaire pour charger les models de CNN qui l'utilisebnt comme metrique.

In [ ]:
from keras import backend as K

""" Pour calculer le score r^2"""
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred), axis=-1) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)), axis=-1)
    return 1.0 - SS_res/(SS_tot + K.epsilon())

Apres avoir defini cette fonction, il suffit d'utiliser la commande suivant pour rendre operationel le model.
``` 
model = keras.models.load_model("NomDuModel.h5", custom_objects={"r2_score": r2_score}, compile=False)
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss=keras.losses.mse, metrics=[r2_score])
```